# ⚙️ Task 10: Model Configuration and Training

## 🎯 Objective
Configure YOLOv11 specifically for our 2-class waste classification task and execute training.

---

## 📚 Training Strategy

### Transfer Learning Approach:
1. Start with pre-trained COCO weights
2. Fine-tune on waste dataset
3. Monitor validation metrics

### ML Rules Applied:
- **Rule #4**: Keep the first model simple
- **Rule #5**: Test infrastructure separately from ML
- **Rule #23**: You are not a typical consumer of your model

In [ ]:
import os
from pathlib import Path
import yaml
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Ultralytics
from ultralytics import YOLO

# Project paths
PROJECT_ROOT = Path(r"D:\het\SELF\RP\YOLO-V11-PRO")
DATA_DIR = PROJECT_ROOT / "data" / "processed"
MODELS_DIR = PROJECT_ROOT / "models"
CONFIGS_DIR = PROJECT_ROOT / "configs"

# Create directories
MODELS_DIR.mkdir(exist_ok=True)
CONFIGS_DIR.mkdir(exist_ok=True)

print("✅ Setup complete!")
print(f"   Data: {DATA_DIR}")
print(f"   Models: {MODELS_DIR}")

---

# Part 1: Waste Classification Config

Our specific configuration for binary waste classification:

In [ ]:
# ============================================================
# WASTE CLASSIFICATION TRAINING CONFIG
# ============================================================

# Dataset YAML
dataset_yaml = DATA_DIR / 'dataset.yaml'

# Model name with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
run_name = f"waste_yolo11n_{timestamp}"

# Training configuration
training_args = {
    # === DATASET ===
    'data': str(dataset_yaml),
    
    # === TRAINING ===
    'epochs': 100,              # Epochs (100 for proper training)
    'patience': 20,             # Early stopping patience
    'batch': 16,                # Batch size
    'imgsz': 640,               # Image size
    
    # === OPTIMIZER ===
    'optimizer': 'AdamW',
    'lr0': 0.01,                # Initial LR
    'lrf': 0.01,                # Final LR = lr0 * lrf
    'momentum': 0.937,
    'weight_decay': 0.0005,
    
    # === AUGMENTATION ===
    'augment': True,
    'hsv_h': 0.015,             # Hue
    'hsv_s': 0.7,               # Saturation
    'hsv_v': 0.4,               # Value
    'degrees': 10,              # Rotation
    'translate': 0.1,           # Translation
    'scale': 0.5,               # Scale
    'shear': 2.0,               # Shear
    'flipud': 0.5,              # Vertical flip
    'fliplr': 0.5,              # Horizontal flip
    'mosaic': 1.0,              # Mosaic
    'mixup': 0.1,               # MixUp
    
    # === OUTPUT ===
    'project': str(MODELS_DIR),
    'name': run_name,
    'exist_ok': True,
    
    # === PERFORMANCE ===
    'device': 0,                # GPU 0
    'workers': 4,
    'amp': True,                # Mixed precision
    
    # === LOGGING ===
    'verbose': True,
    'plots': True,
    'save': True,
    'save_period': 10,          # Save every 10 epochs
}

print("\n📋 Training Configuration:")
print("="*55)
print(f"{'Parameter':<20} {'Value':<30}")
print("-"*55)
for k, v in training_args.items():
    print(f"{k:<20} {str(v):<30}")

In [ ]:
# Save training config
config_file = CONFIGS_DIR / f'{run_name}_config.yaml'
with open(config_file, 'w') as f:
    yaml.dump(training_args, f, default_flow_style=False)
print(f"\n✅ Config saved: {config_file}")

---

# Part 2: Load Model and Train

Now let's train the model!

In [ ]:
# ============================================================
# LOAD YOLOV11 MODEL
# ============================================================

# Load pre-trained YOLOv11 nano
model = YOLO('yolo11n.pt')

print("✅ Model loaded: YOLOv11 Nano")
print("\n📊 Pre-trained on: COCO (80 classes)")
print("🎯 Fine-tuning for: Waste (2 classes)")

In [ ]:
# ============================================================
# TRAIN THE MODEL
# ============================================================

print("\n" + "="*60)
print("🚀 STARTING TRAINING")
print("="*60)
print(f"\n📅 Run: {run_name}")
print(f"📁 Output: {MODELS_DIR / run_name}")
print("\n⏳ Training in progress...\n")

# Train!
results = model.train(**training_args)

print("\n" + "="*60)
print("✅ TRAINING COMPLETE!")
print("="*60)

---

# Part 3: Evaluate Results

After training, let's analyze the results:

In [ ]:
# ============================================================
# ANALYZE TRAINING RESULTS
# ============================================================

# Best model path
best_model_path = MODELS_DIR / run_name / 'weights' / 'best.pt'
last_model_path = MODELS_DIR / run_name / 'weights' / 'last.pt'

print("\n📊 Training Results:")
print("="*50)

if best_model_path.exists():
    print(f"   ✅ Best weights: {best_model_path}")
else:
    print(f"   ⚠️ Best weights not found yet")

if last_model_path.exists():
    print(f"   ✅ Last weights: {last_model_path}")

In [ ]:
# Load training plots if available
results_dir = MODELS_DIR / run_name

plots = [
    'results.png',
    'confusion_matrix.png',
    'F1_curve.png',
    'PR_curve.png',
]

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

for ax, plot_name in zip(axes.flat, plots):
    plot_path = results_dir / plot_name
    if plot_path.exists():
        img = plt.imread(plot_path)
        ax.imshow(img)
        ax.set_title(plot_name.replace('.png', '').replace('_', ' '))
    else:
        ax.text(0.5, 0.5, f'{plot_name}\n(Not available)', ha='center', va='center')
    ax.axis('off')

plt.suptitle('📈 Training Results', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'docs' / 'assets' / 'training_results.png', dpi=150)
plt.show()

In [ ]:
# ============================================================
# VALIDATE MODEL
# ============================================================

if best_model_path.exists():
    print("\n🔍 Validating best model...")
    
    best_model = YOLO(str(best_model_path))
    val_results = best_model.val(data=str(dataset_yaml))
    
    print("\n📊 Validation Metrics:")
    print("="*50)
    print(f"   mAP@50:    {val_results.box.map50:.4f}")
    print(f"   mAP@50-95: {val_results.box.map:.4f}")
    print(f"   Precision: {val_results.box.mp:.4f}")
    print(f"   Recall:    {val_results.box.mr:.4f}")

In [ ]:
# ============================================================
# TEST INFERENCE
# ============================================================

from PIL import Image

# Get test images
test_images = list((DATA_DIR / 'images' / 'val').glob('*.jpg'))[:6]

if best_model_path.exists() and test_images:
    best_model = YOLO(str(best_model_path))
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle('🎯 Model Predictions on Validation Set', fontsize=14, fontweight='bold')
    
    for ax, img_path in zip(axes.flat, test_images):
        # Run inference
        results = best_model.predict(source=str(img_path), conf=0.25, save=False, verbose=False)
        
        # Plot
        result_img = results[0].plot()
        ax.imshow(result_img)
        ax.set_title(img_path.name[:25])
        ax.axis('off')
    
    plt.tight_layout()
    plt.savefig(PROJECT_ROOT / 'docs' / 'assets' / 'model_predictions.png', dpi=150)
    plt.show()

---

# Part 4: Export Model

Export for deployment:

In [ ]:
# Export to different formats
if best_model_path.exists():
    best_model = YOLO(str(best_model_path))
    
    print("\n📦 Exporting model...")
    
    # Export to ONNX (cross-platform)
    onnx_path = best_model.export(format='onnx')
    print(f"   ✅ ONNX: {onnx_path}")
    
    # The PyTorch model is already saved
    print(f"   ✅ PyTorch: {best_model_path}")

## 📝 Summary

### Training Complete:
- ✅ Model trained on waste dataset
- ✅ Best weights saved
- ✅ Validation metrics computed
- ✅ Model exported

### Key Files:
- `models/{run_name}/weights/best.pt` - Best model
- `models/{run_name}/results.png` - Training curves
- `models/{run_name}/confusion_matrix.png` - Confusion matrix

### Next: Task 11 - Evaluation & Deployment

In [ ]:
print("\n" + "="*60)
print("✅ TASK 10 COMPLETE: Model Training")
print("="*60)
print("\n📋 Accomplished:")
print("   ✓ Training configuration set")
print("   ✓ Model trained on waste dataset") 
print("   ✓ Training metrics plotted")
print("   ✓ Model validated")
print("   ✓ Inference tested")
print("   ✓ Model exported")
print("\n➡️ Ready for Deployment!")